In [1]:
# Idea: create a CSV that has fields timestamp,queued,util_gpu1,util_mem1,util_gpu2,util_mem2,util_gpu3,util_mem3
# Use Cell 6 convert the files in the results/ folder into the pickel file that will be read by the images
# notebook
def addToOutput(timestamp, elem):
    if timestamp not in output:
        output[timestamp] = {}
    for key in elem:
        output[timestamp][key] = elem[key]

def read2GPUs(filename):
    with open(filename, "r") as f:
        elem = {}
        while True:
            time_l = f.readline()
            if not time_l:
                break
            gpu1_l = f.readline()
            gpu2_l = f.readline()
            split1 = gpu1_l.split(",")
            split2 = gpu2_l.split(",")
            # gpu_bus_id,utilization.gpu,utilization.memory
            newO = {
                #'timestamp': round(int(time_l)/1000)*1000 # rounded to the next second
                'util_gpu1': int(split1[1]),
                'util_mem1': int(split1[2]),
                'util_gpu2': int(split2[1]),
                'util_mem2': int(split2[2])
            }
            addToOutput(round(int(time_l)/1000)*1000, newO)

def read1GPU(filename):
    with open(filename, "r") as f:
        elem = {}
        while True:
            time_l = f.readline()
            if not time_l:
                break
            gpu1_l = f.readline()
            split1 = gpu1_l.split(",")
            # gpu_bus_id,utilization.gpu,utilization.memory
            newO = {
                #'timestamp': round(int(time_l)/1000)*1000 # rounded to the next second
                'util_gpu3': int(split1[1]),
                'util_mem3': int(split1[2])
            }
            addToOutput(round(int(time_l)/1000)*1000, newO)

def readQueue(filename):
    with open(filename, "r") as f:
        elem = {}
        f.readline() # Ignore the first line
        while True:
            line = f.readline()
            if not line:
                break
            split = line.replace("\"", "").replace("\n", "").split(";")
            # gpu_bus_id,utilization.gpu,utilization.memory
            newO = {
                'queued': int(split[1]),
            }
            addToOutput(round(int(split[0])/1000)*1000, newO)

In [6]:
# 2021-07-13T18:02:21_inv_singleServerAllVPU_120000_1.0_600000_120000_4.0
output = {}
readQueue("../../../long-results/2021-07-13T18:02:21_queue_singleServerAllVPU.csv")
read1GPU("../../../long-results/2021-07-13T18:02:21_gpuUsuage_second.csv")
read2GPUs("../../../long-results/2021-07-13T18:02:21_gpuUsuage.csv")
output

FileNotFoundError: [Errno 2] No such file or directory: '../../../long-results/2021-07-13T18:02:21_gpuUsuage_second.csv'

In [5]:
import pandas as pd
import collections
ordered_o = collections.OrderedDict(sorted(output.items()))
cleaned_o = []
for key, v in ordered_o.items():
    clean = {}
    clean['time'] = key
    for e_key in output[key]:
        clean[e_key] = output[key][e_key]
    cleaned_o.append(clean)
df = pd.DataFrame(cleaned_o)
pd.to_pickle(df, "../../../long-results/2021-07-13T18:02:21_stats.pkl")
df

,time,util_gpu1,util_mem1,util_gpu2,util_mem2,queued
0,1627912050000,1.0,0.0,0.0,0.0,NaN
1,1627912051000,1.0,0.0,0.0,0.0,NaN
2,1627912053000,1.0,0.0,0.0,0.0,NaN
3,1627912054000,1.0,0.0,0.0,0.0,NaN
4,1627912056000,1.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...
1294,1627913346000,1.0,0.0,0.0,0.0,0.0
1295,1627913347000,1.0,0.0,0.0,0.0,0.0
1296,1627913348000,NaN,NaN,NaN,NaN,0.0
1297,1627913349000,1.0,0.0,0.0,0.0,NaN


In [4]:
df['time'].min()

1626716704000

In [5]:
df['time'].max()

1626717751000

In [6]:
df['util_gpu3'].max()

91.0

In [7]:
df.loc[df['util_gpu2'] != 0]

,time,util_gpu3,util_mem3,util_gpu1,util_mem1,util_gpu2,util_mem2,queued
0,1626716704000,0.0,0.0,NaN,NaN,NaN,NaN,NaN
7,1626716714000,NaN,NaN,NaN,NaN,NaN,NaN,55.0
10,1626716717000,30.0,3.0,NaN,NaN,NaN,NaN,138.0
12,1626716719000,53.0,15.0,85.0,26.0,10.0,2.0,124.0
13,1626716720000,24.0,7.0,100.0,31.0,100.0,21.0,111.0
...,...,...,...,...,...,...,...,...
1035,1626717742000,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1038,1626717745000,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1041,1626717748000,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1043,1626717750000,NaN,NaN,NaN,NaN,NaN,NaN,0.0
